<a href="https://colab.research.google.com/github/it-ces/Thesis-bankrupt/blob/main/Notebooks/Models(Thesis).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
################################
#         LIBRARIES            #
################################

## For processing data
import os
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split

#To describe

from tableone  import TableOne
from scipy import stats


## To modeling
# Models
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

# split and grid
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

## Assesment
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support, confusion_matrix, precision_score, recall_score, roc_auc_score


In [2]:
################################
#   Our Functions   (I)       #
################################
def percentage_nulls(df):
    """
    This function returns a dictionary with the column and 
    the porcentage of missing values
    """
    N_rows = df.shape[0]
    vars_ = {}
    for var in df_consumo.columns:
        vars_[var]=(df_consumo[var].isnull().sum() / N_rows)
    return vars_


def removing_cols(df, percentages_dict, threshold):
    """
    Receive a dictionary with the percatege of missing of each varaible and drop them
    according to the threshold defined
    """
    df_ = df.copy()
    for var in percentages_dict:
        if percentages_dict[var] > threshold:
            df_.drop(columns = [var], inplace=True)
    return df_


def breakdown_vars(df):
    """
    This function allow us categorize accodign to numerical or not
    """
    categorial = []
    nonormal = []
    normal = []
    for t in df.columns:
        if df[t].dtypes=="object" or df[t].dtypes.name=='category':
            categorial.append(t)
        if df[t].dtypes=="int64" or df[t].dtypes=="float64":
                n,p = stats.shapiro(df[t])
                if p<0.05:
                    nonormal.append(t)
                else: 
                    normal.append(t)
    return categorial, nonormal, normal


In [3]:
################################
#   Our Functions   (II)       #
################################
def std_z(nums, df_, event):
    """
    standardizing nums(numerical) variables
    """
    df = df_.copy()
    for col in nums:
        if col != event:
            df[col] = (df[col] - df[col].mean())/df[col].std()
    return df


def Xy(df_,target):
    """
    Split the data in X,y to ML implementations
    """
    df = df_.copy()
    X = df.loc[ : , df.columns != target]
    y = df[target]
    return X,y


def dummies_ohe(df_,cats):
    """
    Returns a dataframe with dummies,and dropped the categorical in original
    the cats arguments receive the cats to transform.
    """
    df = df_.copy()
    df.reset_index(drop=True, inplace=True)
    ohe = OneHotEncoder(drop='first',handle_unknown='ignore', sparse_output=False)
    dummies = pd.DataFrame(ohe.fit_transform(df[cats]))
    dummies.columns = ohe.get_feature_names_out()  #Names ohe.get_feature_names_out()-> all dummies
    df.drop(columns=cats, inplace=True)
    df = pd.concat([df,dummies], axis=1)
    return df



def varInEvent(time_base, time_last, time_target, prefix, var_time_event, df_, lag=1, mean=True):
    """ put time_target  = 0 if you use mean=True"""
    df = df_.copy()
    df['var*'] = np.nan
    for firm in df.index:
        time_ocurrence = df.loc[firm, var_time_event]
        if time_ocurrence - lag >= time_base:
            df.loc[firm, 'var*'] = df.loc[firm, prefix + str(int(time_ocurrence-lag))]
        if mean==True:
            to_mean =  [prefix+str(int(time)) for time in range(time_base, time_last+1)]
            df['var*'] =np.where(df[var_time_event]==0, df[to_mean].mean(axis=1), df['var*'])
        else:
            df['var*'] =np.where(df[var_time_event]==0, df[prefix+str(time_target-lag)], df['var*'])
    return df['var*']


In [4]:
################################
#   Models with Grid search    #
################################


# Grid search hyperparameters for a LOGISTIC REGRESSION model
def grid_lr(X_train, y_train):
    model = LogisticRegression(random_state=666, max_iter=1000)
    class_weight =  [{0:0.5, 1:0.5}, {0:0.1, 1:0.9}, {0:0.2, 1:0.8}]
    solvers = ['liblinear']
    penalty = ['l2','l1']
    c_values = [ 10, 1.0, 0.1, 0.01, 0.001, ]
    grid = dict(solver=solvers,penalty=penalty,C=c_values, class_weight= class_weight)
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv,
                           scoring='f1',error_score='raise')
    grid_result = grid_search.fit(X_train, y_train)
    return  grid_result.best_estimator_


# Grid search hyperaparmeters for DECISION TREE model
def grid_dt(X_train, y_train):
    model = DecisionTreeClassifier(random_state=666)
    class_weight =  [{0:0.5, 1:0.5}, {0:0.1, 1:0.9}, {0:0.2, 1:0.8}]
    max_depth = None,
    min_samples_leaf = [5, 10, 20, 50, 100]
    criterion  = ["gini", "entropy"]
    grid = dict(class_weight=class_weight, max_depth=max_depth, min_samples_leaf=min_samples_leaf, criterion=criterion)
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
    grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv,
                           scoring='f1',error_score=0)
    grid_result = grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_



# Grid search hyperparameters for MULTILAYER PERCEPTRON model
def grid_MLP(X_train, y_train):
    model = MLPClassifier(random_state=1, max_iter=300)
    hidden_layer_sizes = [(8,), (100,)]
    activation =  ['tanh', 'relu', 'logistic']
    solver =  ['sgd', 'adam']
    alpha  = [0.0001, 0.05]
    #'learning_rate': ['constant','adaptive'], }
    grid = dict(hidden_layer_sizes=hidden_layer_sizes, activation= activation, solver= solver)
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv,
                           scoring='f1',error_score='raise')
    grid_result = grid_search.fit(X_train, y_train)
    return  grid_result.best_estimator_

#######################################################
#                                                     #
#                RATIOS                               #
#                                                     #   
#######################################################


def ratios(df_,):
    df = df_.copy()
  # Marge de ganancia bruta (MGB)
  # Ganancia bruta / ingresos de actividades ordianrias
    df['MGB'] = df['Ganancia bruta'] / df['Ingresos de actividades ordinarias']
  # Remember that ganancia bruta is:
    df['Ingresos de actividades ordinarias'] - df['Costo de ventas']
  ## Margen de ganancia Neta (MGN)
  #df['Ganancia (pérdida) antes de impuestos'] - df['Ingreso (gasto) por impuestos'] #esto es el resultado del periodo
    #df[ 'Ganancia (pérdida)'] # que en la base resultado del periodo es esta variable
    df['MGN'] = df[ 'Ganancia (pérdida)'] /  df['Ingresos de actividades ordinarias']
  # Rendimiento del patrimonio (ROE)
    df['ROE'] = df['Ganancia (pérdida)']/ df['Patrimonio total']
  # Rendimiento del Activo (ROA)
    df['ROA'] = df['Ganancia (pérdida)']/ df['Total de activos']
  ## NIVEL DE ENDEUDAMIENTO (NE)
    df['NE'] = df['Total pasivos'] / df['Total de activos']
  ## Concenttración de Pasivos a corto plazo (PCP)
    df['PCP'] = df['Pasivos corrientes totales'] / df['Total pasivos']
  ## Endeudamiento Financiero (EF)
  ## Crear primero otros pasivos financieros
   # df['Otros pasivos financieros'] = df['Otros pasivos financieros corrientes'] + df['Otros activos no financieros corrientes']
    #df['EF'] = df['Otros pasivos financieros'] /  df['Ingresos de actividades ordinarias']
  ## Impacto de la carga financiera (IF)
    #df['IF'] = df['Costos financieros']/ df['Ingresos de actividades ordinarias']
  ## ALTMAN
    df['Ax1'] = (df['Activos corrientes totales'] - df['Pasivos corrientes totales'])/df['Total de activos']
    df['Ax2'] = (df['Ganancias acumuladas'] / df['Total de activos'])
    return df

#######################################################
#                                                     #
#        NULL variables and imputation                #
#                                                     #   
#######################################################


def complete_vars(df):
    """
    percentage of columns with information
    """
    df_ = df.copy()
    N_cols = df_.shape[1]
    return 1 - df_.apply(lambda x: pd.isnull(x)).sum(axis=1)/N_cols


def imputation_mean(df_, target_var, condition_var, condition_value):
    """
    target_var: The variable that we want imputate
    condicion_var: The variable that we want uses to stratify the imputation 
    """
    df = df_.copy()
    indices = df[df[condition_var]==condition_value].index
    mean_to_replace = df[df[condition_var]==condition_value][target_var].mean()
    df.loc[indices, target_var] = df[df[condition_var]==condition_value][target_var].replace(np.nan, mean_to_replace)
    return df[target_var]

In [5]:
#######################################################
#                                                     #
#                POOLED DATABASE                      #
#                                                     #   
#######################################################
url = "https://raw.githubusercontent.com/it-ces/Thesis-bankrupt/main/Datasets/database-to-model.csv"
df = pd.read_csv(url)

#df['Patrimonio total'] = varInEvent(2016, 2018, 0, 'Patrimonio total-', 'time-event', df)

## To check the construction of variables adecuately
#patrimonios = ['Patrimonio total-'+str(year) for year in range(2016,2019)]
#df[patrimonios + ['Patrimonio total', 'event' ,'time-event','NIT'] ].to_excel('revisar.xlsx')



VARS = ['Ganancia bruta', 'Ganancia (pérdida)','Ingresos de actividades ordinarias' , 'Costo de ventas', 'Patrimonio total',
     'Total pasivos', 'Total de activos', 'Ganancias acumuladas',  'Pasivos corrientes totales',  'Activos corrientes totales']

###################################
# Activate the following          #
# loop to reconstruc the database #
#                                 #
###################################

#for var in VARS:
    #df[var] = varInEvent(2016, 2018, 0, var+'-', 'time-event', df)
#df.to_csv("Datapooled.csv")

#for var in VARS:
    #df[var] = varInEvent(2016, 2018, 0, var+'-', 'time-event', df, lag=2)
#df.to_csv("Datapooled2.csv")

In [6]:
######################
# POOLED MODEL lag=1##
#####################
df_train = pd.read_csv("Datapooled.csv")
df_train = df_train[VARS+['NIT', 'event', 'time-event',  'Clasificación Industrial Internacional Uniforme Versión 4 A.C']]
print(df_train[df_train['event']==1].info())
df_train['complete-vars'] = complete_vars(df_train) #1 is that have all variables!
df_train =  df_train[df_train['complete-vars']==1] #filtering firms that have not financial information 
print(df_train[df_train['event']==1].info())
df_train = ratios(df_train)
predictors =[ 'MGB', 'MGN', 'ROE','ROA', 'NE', 'PCP',  'Ax1', 'Ax2',
             'Clasificación Industrial Internacional Uniforme Versión 4 A.C']

print(df_train[df_train['event']==1].info())
df_train.replace([np.inf,-np.inf], np.nan, inplace=True)
df_train.dropna(inplace=True)

#*
df_tab = df_train = df_train[[ 'event' , 'time-event'] + predictors].copy()
df_tab['time-event'] = df_tab['time-event'].astype('str')
#*



###################
#   TABLE ONE     #
###################
df_tab.reset_index(drop=True, inplace=True) # It is necessary for the next function
cat, nonormal, normal  = breakdown_vars(df_tab)
#df = std_z(nonormal + normal, df, 'event')  # Standardize the variables (WORK) study this properties
mytable = TableOne(df_tab, categorical=cat, nonnormal=nonormal,  groupby='event', pval=True, decimals=2)
display(mytable)
mytable.to_latex("tableonelag1.tex")




###################
## Modelling Xy  ##
###################
df_train = df_train[[ 'event'] + predictors]
df = df_train.copy() 
df.reset_index(drop=True, inplace=True) # It is necessary for the next function
cat, nonormal, normal  = breakdown_vars(df)
df = dummies_ohe(df_train,cat)
X,y = Xy(df,'event')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True, random_state = 666, stratify=y)


print(y_train.value_counts())
print(y_test.value_counts())



### Logit model
best_model = grid_lr(X_train, y_train)
preds = best_model.predict(X_test)
print(classification_report(y_test, preds))  # recall igual a sensibilidad
logit  = pd.DataFrame(classification_report(y_test, preds , output_dict=True)).iloc[:,0:2]





### Decision tree
best_model = grid_dt(X_train, y_train)
preds = best_model.predict(X_test)
print(classification_report(y_test, preds))  # recall igual a sensibilidad
tree = pd.DataFrame(classification_report(y_test, preds , output_dict=True)).iloc[:,0:2]



### Modelling perceptron
best_model = grid_MLP(X_train, y_train)
preds = best_model.predict(X_test)
print(classification_report(y_test, preds))  # recall igual a sensibilidad
MLP =  pd.DataFrame(classification_report(y_test, preds , output_dict=True)).iloc[:,0:2]



models_tab = pd.concat([logit, tree, MLP], axis=1)
cols_names =  pd.MultiIndex.from_tuples([('Logistic Regression','No-Default'),("Logistic Regression",'Deafult'),
              ("Decision Tree  ",'No-Default'),('Decision Tree  ','Default'),
              ("Multilayer Perceptron",'No-Default'),('Multilayer Perceptron','Default')])
models_tab.columns  = cols_names
models_tab = models_tab.style.set_table_styles([
   {'selector': 'th','props': [('text-align', 'center')]}]).format(precision=2)
models_tab.to_latex("lag1results.tex")

display(models_tab)


##################################
# Modelling with undersampling   #
##################################


from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=1234)
X_res, y_res = rus.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, shuffle = True, random_state = 666, stratify=y_res)



##### Results undersampling


### Logit model
best_model = grid_lr(X_train, y_train)
preds = best_model.predict(X_test)
print(classification_report(y_test, preds))  # recall igual a sensibilidad
logit  = pd.DataFrame(classification_report(y_test, preds , output_dict=True)).iloc[:,0:2]


### Decision tree
best_model = grid_dt(X_train, y_train)
preds = best_model.predict(X_test)
print(classification_report(y_test, preds))  # recall igual a sensibilidad
tree = pd.DataFrame(classification_report(y_test, preds , output_dict=True)).iloc[:,0:2]


### Modelling perceptron
best_model = grid_MLP(X_train, y_train)
preds = best_model.predict(X_test)
print(classification_report(y_test, preds))  # recall igual a sensibilidad
MLP =  pd.DataFrame(classification_report(y_test, preds , output_dict=True)).iloc[:,0:2]





models_tab = pd.concat([logit, tree, MLP], axis=1)
cols_names =  pd.MultiIndex.from_tuples([('Logistic Regression','No-Default'),("Logistic Regression",'Deafult'),
              ("Decision Tree  ",'No-Default'),('Decision Tree  ','Default'),
              ("Multilayer Perceptron",'No-Default'),('Multilayer Perceptron','Default')])
models_tab.columns  = cols_names
models_tab = models_tab.style.set_table_styles([
   {'selector': 'th','props': [('text-align', 'center')]}]).format(precision=2)
models_tab.to_latex("underlag1results.tex")
models_tab


<class 'pandas.core.frame.DataFrame'>
Index: 771 entries, 4 to 26499
Data columns (total 14 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   Ganancia bruta                                                 353 non-null    float64
 1   Ganancia (pérdida)                                             353 non-null    float64
 2   Ingresos de actividades ordinarias                             353 non-null    float64
 3   Costo de ventas                                                330 non-null    float64
 4   Patrimonio total                                               353 non-null    float64
 5   Total pasivos                                                  353 non-null    float64
 6   Total de activos                                               353 non-null    float64
 7   Ganancias acumuladas                                           35

C:\Users\iván\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


Grouped by event                                                                
                                                                                                                                                                                                              Missing           Overall               0.0                 1.0 P-Value
n                                                                                                                                                                                                                                 16806             16488                 318        
time-event, n (%)                                                    0.0                                                                                                                                            0     16488 (98.11)    16488 (100.00)                      <0.001
                                                                     2017.0                                                                                                                                                   99 (0.59)                            99 (31.13)        
                                                                     2018.0                                                                                                                                                   96 (0.57)                            96 (30.19)        
                                                                     2019.0                                                                                                                                                  123 (0.73)                           123 (38.68)        
MGB, median [Q1,Q3]                                                                                                                                                                                                 0  0.31 [0.18,0.56]  0.32 [0.18,0.56]    0.26 [0.14,0.41]  <0.001
MGN, median [Q1,Q3]                                                                                                                                                                                                 0  0.03 [0.00,0.08]  0.03 [0.00,0.08]  -0.02 [-0.18,0.03]  <0.001
ROE, median [Q1,Q3]                                                                                                                                                                                                 0  0.07 [0.01,0.16]  0.07 [0.01,0.16]   0.01 [-0.18,0.09]  <0.001
ROA, median [Q1,Q3]                                                                                                                                                                                                 0  0.03 [0.00,0.06]  0.03 [0.00,0.07]  -0.01 [-0.09,0.01]  <0.001
NE, median [Q1,Q3]                                                                                                                                                                                                  0  0.53 [0.32,0.72]  0.52 [0.32,0.72]    0.74 [0.59,0.88]  <0.001
PCP, median [Q1,Q3]                                                                                                                                                                                                 0  0.75 [0.45,0.98]  0.75 [0.45,0.98]    0.53 [0.28,0.87]  <0.001
Ax1, median [Q1,Q3]                                                                                                                                                                                                 0  0.22 [0.04,0.43]  0.22 [0.04,0.43]   0.09 [-0.08,0.28]  <0.001
Ax2, median [Q1,Q3]                                                                                                                                                                                                 0  0.19 [0.04,0.40]  0.19 [0.05,0.40]   0.04 [-0.12,0.18]  <0.001
Clasificación Industrial In

C:\Users\iván\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


event
0.0    13190
1.0      254
Name: count, dtype: int64
event
0.0    3298
1.0      64
Name: count, dtype: int64
              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99      3298
         1.0       0.10      0.03      0.05        64

    accuracy                           0.98      3362
   macro avg       0.54      0.51      0.52      3362
weighted avg       0.96      0.98      0.97      3362

              precision    recall  f1-score   support

         0.0       0.98      0.94      0.96      3298
         1.0       0.06      0.20      0.09        64

    accuracy                           0.92      3362
   macro avg       0.52      0.57      0.52      3362
weighted avg       0.97      0.92      0.94      3362

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      3298
         1.0       0.16      0.05      0.07        64

    accuracy                           0.98      3362
   macro avg  

              precision    recall  f1-score   support

         0.0       0.64      0.73      0.68        64
         1.0       0.69      0.58      0.63        64

    accuracy                           0.66       128
   macro avg       0.66      0.66      0.65       128
weighted avg       0.66      0.66      0.65       128

              precision    recall  f1-score   support

         0.0       0.72      0.45      0.56        64
         1.0       0.60      0.83      0.70        64

    accuracy                           0.64       128
   macro avg       0.66      0.64      0.63       128
weighted avg       0.66      0.64      0.63       128

              precision    recall  f1-score   support

         0.0       0.72      0.75      0.73        64
         1.0       0.74      0.70      0.72        64

    accuracy                           0.73       128
   macro avg       0.73      0.73      0.73       128
weighted avg       0.73      0.73      0.73       128



C:\Users\iván\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [7]:
##########################
# Estimation with lag=2  #
##########################

df_train = pd.read_csv("Datapooled2.csv")
df_train = df_train[VARS+['NIT', 'event', 'time-event',  'Clasificación Industrial Internacional Uniforme Versión 4 A.C']]
print(df_train[df_train['event']==1].info())
df_train['complete-vars'] = complete_vars(df_train) #1 is that have all variables!
df_train =  df_train[df_train['complete-vars']==1] #filtering firms that have not financial information 
print(df_train[df_train['event']==1].info())
df_train = ratios(df_train)
predictors =[ 'MGB', 'MGN', 'ROE','ROA', 'NE', 'PCP',  'Ax1', 'Ax2',
             'Clasificación Industrial Internacional Uniforme Versión 4 A.C']

print(df_train[df_train['event']==1].info())
df_train.replace([np.inf,-np.inf], np.nan, inplace=True)
df_train.dropna(inplace=True)

#*
df_tab = df_train = df_train[[ 'event' , 'time-event'] + predictors].copy()
df_tab['time-event'] = df_tab['time-event'].astype('str')
#*


###################
#   TABLE ONE     #
###################
df_tab.reset_index(drop=True, inplace=True) # It is necessary for the next function
cat, nonormal, normal  = breakdown_vars(df_tab)
#df = std_z(nonormal + normal, df, 'event')  # Standardize the variables (WORK) study this properties
mytable = TableOne(df_tab, categorical=cat, nonnormal=nonormal,  groupby='event', pval=True, decimals=2)
display(mytable)
mytable.to_latex("tableonelag2.tex")



###################
## Modelling Xy  ##
###################
df_train = df_train[[ 'event'] + predictors]
df = df_train.copy() 
df.reset_index(drop=True, inplace=True) # It is necessary for the next function
cat, nonormal, normal  = breakdown_vars(df)
df = dummies_ohe(df_train,cat)
X,y = Xy(df,'event')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True, random_state = 666, stratify=y)


print(y_train.value_counts())
print(y_test.value_counts())






### Logit model
best_model = grid_lr(X_train, y_train)
preds = best_model.predict(X_test)
print(classification_report(y_test, preds))  # recall igual a sensibilidad
logit  = pd.DataFrame(classification_report(y_test, preds , output_dict=True)).iloc[:,0:2]



### Decision tree
best_model = grid_dt(X_train, y_train)
preds = best_model.predict(X_test)
print(classification_report(y_test, preds))  # recall igual a sensibilidad
tree = pd.DataFrame(classification_report(y_test, preds , output_dict=True)).iloc[:,0:2]



### Modelling perceptron
best_model = grid_MLP(X_train, y_train)
preds = best_model.predict(X_test)
print(classification_report(y_test, preds))  # recall igual a sensibilidad
MLP =  pd.DataFrame(classification_report(y_test, preds , output_dict=True)).iloc[:,0:2]





models_tab = pd.concat([logit, tree, MLP], axis=1)
cols_names =  pd.MultiIndex.from_tuples([('Logistic Regression','No-Default'),("Logistic Regression",'Deafult'),
              ("Decision Tree  ",'No-Default'),('Decision Tree  ','Default'),
              ("Multilayer Perceptron",'No-Default'),('Multilayer Perceptron','Default')])
models_tab.columns  = cols_names
models_tab = models_tab.style.set_table_styles([
   {'selector': 'th','props': [('text-align', 'center')]}]).format(precision=2)
models_tab.to_latex("lag2results.tex")

display(models_tab)

##################################
# Modelling with undersampling   #
##################################

from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=1234)
X_res, y_res = rus.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, shuffle = True, random_state = 666, stratify=y_res)



##### Results undersampling



### Logit model
best_model = grid_lr(X_train, y_train)
preds = best_model.predict(X_test)
print(classification_report(y_test, preds))  # recall igual a sensibilidad
logit  = pd.DataFrame(classification_report(y_test, preds , output_dict=True)).iloc[:,0:2]


### Decision tree
best_model = grid_dt(X_train, y_train)
preds = best_model.predict(X_test)
print(classification_report(y_test, preds))  # recall igual a sensibilidad
tree = pd.DataFrame(classification_report(y_test, preds , output_dict=True)).iloc[:,0:2]


### Modelling perceptron
best_model = grid_MLP(X_train, y_train)
preds = best_model.predict(X_test)
print(classification_report(y_test, preds))  # recall igual a sensibilidad
MLP =  pd.DataFrame(classification_report(y_test, preds , output_dict=True)).iloc[:,0:2]


models_tab = pd.concat([logit, tree, MLP], axis=1)
cols_names =  pd.MultiIndex.from_tuples([('Logistic Regression','No-Default'),("Logistic Regression",'Deafult'),
              ("Decision Tree  ",'No-Default'),('Decision Tree  ','Default'),
              ("Multilayer Perceptron",'No-Default'),('Multilayer Perceptron','Default')])
models_tab.columns  = cols_names
models_tab = models_tab.style.set_table_styles([
   {'selector': 'th','props': [('text-align', 'center')]}]).format(precision=2)
models_tab.to_latex("underlag2results.tex")
models_tab

<class 'pandas.core.frame.DataFrame'>
Index: 771 entries, 4 to 26499
Data columns (total 14 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   Ganancia bruta                                                 211 non-null    float64
 1   Ganancia (pérdida)                                             211 non-null    float64
 2   Ingresos de actividades ordinarias                             211 non-null    float64
 3   Costo de ventas                                                202 non-null    float64
 4   Patrimonio total                                               211 non-null    float64
 5   Total pasivos                                                  211 non-null    float64
 6   Total de activos                                               211 non-null    float64
 7   Ganancias acumuladas                                           21

C:\Users\iván\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


Grouped by event                                                               
                                                                                                                                                                                                              Missing           Overall               0.0                1.0 P-Value
n                                                                                                                                                                                                                                 16684             16488                196        
time-event, n (%)                                                    0.0                                                                                                                                            0     16488 (98.83)    16488 (100.00)                     <0.001
                                                                     2018.0                                                                                                                                                   91 (0.55)                           91 (46.43)        
                                                                     2019.0                                                                                                                                                  105 (0.63)                          105 (53.57)        
MGB, median [Q1,Q3]                                                                                                                                                                                                 0  0.31 [0.18,0.56]  0.32 [0.18,0.56]   0.27 [0.17,0.43]   0.002
MGN, median [Q1,Q3]                                                                                                                                                                                                 0  0.03 [0.00,0.08]  0.03 [0.00,0.08]  0.01 [-0.05,0.04]  <0.001
ROE, median [Q1,Q3]                                                                                                                                                                                                 0  0.07 [0.01,0.16]  0.07 [0.01,0.16]  0.02 [-0.06,0.09]  <0.001
ROA, median [Q1,Q3]                                                                                                                                                                                                 0  0.03 [0.00,0.06]  0.03 [0.00,0.07]  0.01 [-0.03,0.03]  <0.001
NE, median [Q1,Q3]                                                                                                                                                                                                  0  0.52 [0.32,0.72]  0.52 [0.32,0.72]   0.67 [0.53,0.79]  <0.001
PCP, median [Q1,Q3]                                                                                                                                                                                                 0  0.75 [0.45,0.98]  0.75 [0.45,0.98]   0.55 [0.28,0.84]  <0.001
Ax1, median [Q1,Q3]                                                                                                                                                                                                 0  0.22 [0.04,0.43]  0.22 [0.04,0.43]  0.14 [-0.01,0.31]  <0.001
Ax2, median [Q1,Q3]                                                                                                                                                                                                 0  0.19 [0.04,0.40]  0.19 [0.05,0.40]  0.11 [-0.02,0.27]  <0.001
Clasificación Industrial Internacional Uniforme Versión 4 A.C, n (%) Actividades artísticas, de entretenimiento y recreación                                                                                        0         86 (0.52)         86 (0.52)                      0.029
                                          

C:\Users\iván\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


event
0.0    13190
1.0      157
Name: count, dtype: int64
event
0.0    3298
1.0      39
Name: count, dtype: int64
              precision    recall  f1-score   support

         0.0       0.98      0.05      0.10      3298
         1.0       0.01      0.90      0.02        39

    accuracy                           0.06      3337
   macro avg       0.49      0.47      0.06      3337
weighted avg       0.97      0.06      0.10      3337

              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98      3298
         1.0       0.01      0.03      0.02        39

    accuracy                           0.96      3337
   macro avg       0.50      0.50      0.50      3337
weighted avg       0.98      0.96      0.97      3337

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      3298
         1.0       0.00      0.00      0.00        39

    accuracy                           0.99      3337
   macro avg  

              precision    recall  f1-score   support

         0.0       1.00      0.12      0.22        40
         1.0       0.53      1.00      0.69        39

    accuracy                           0.56        79
   macro avg       0.76      0.56      0.46        79
weighted avg       0.77      0.56      0.45        79

              precision    recall  f1-score   support

         0.0       0.79      0.38      0.51        40
         1.0       0.58      0.90      0.71        39

    accuracy                           0.63        79
   macro avg       0.69      0.64      0.61        79
weighted avg       0.69      0.63      0.61        79

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        40
         1.0       0.49      1.00      0.66        39

    accuracy                           0.49        79
   macro avg       0.25      0.50      0.33        79
weighted avg       0.24      0.49      0.33        79



C:\Users\iván\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\iván\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\iván\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [8]:
MLP

,0.0,1.0
precision,0.0,0.493671
recall,0.0,1.000000
f1-score,0.0,0.661017
support,40.0,39.000000


In [9]:
logit

,0.0,1.0
precision,1.000000,0.527027
recall,0.125000,1.000000
f1-score,0.222222,0.690265
support,40.000000,39.000000


In [10]:
tree

,0.0,1.0
precision,0.789474,0.583333
recall,0.375000,0.897436
f1-score,0.508475,0.707071
support,40.000000,39.000000
